In [10]:
!docker stop ipfs_host

ERROR: The Compose file './docker-compose.yml' is invalid because:
Unsupported config option for services.networks: 'backend'


In [11]:
!docker rm ipf_host

ERROR: The Compose file './docker-compose.yml' is invalid because:
Unsupported config option for services.networks: 'backend'


In [12]:
!docker run -d --name ipfs_host2 -v $ipfs_staging:/export -v $ipfs_data:/data/ipfs -p 4001:4001 -p 4001:4001/udp -p 127.0.0.1:8080:8080 -p 127.0.0.1:5001:5001 ipfs/go-ipfs:latest

539f3618e5ff0b0a85e7d61b47c4aeeb89876547f596fe3bad72779d35980a25


In [1]:
!mkdir /tmp/posts
!mkdir /tmp/comments

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import json
import substrateinterface
from substrateinterface import SubstrateInterface, Keypair
from substrateinterface.exceptions import SubstrateRequestException
from substrate_helpers import make_call, addSchema, get_msa_id, \
            get_signature, create_msa_with_delegator, mint_votes, mint_user, get_schemas_from_pattern, \
            get_content_from_schemas, get_user, follow_user, add_delegate

/home/chia/miniconda3/lib/python3.8/site-packages/ipfshttpclient/client/__init__.py:73: VersionMismatch: Unsupported daemon version '0.13.0' (not in range: 0.5.0 ≤ … < 0.9.0)
  warnings.warn(exceptions.VersionMismatch(version, minimum, maximum))


In [23]:
bob = Keypair.create_from_uri('//Bob')
substrate = SubstrateInterface(
    url="ws://127.0.0.1:9944",
    ss58_format=42,
    type_registry_preset='kusama'
)

In [24]:
bob_msa_id = get_msa_id(bob, create=True)

In [25]:
bob_msa_id

1

# Make schemas

In [36]:
post_schema = "category STRING,title STRING,body STRING,url STRING,is_nsfw NUMERIC"
post_schemaId = addSchema(post_schema, wait_for_inclusion=True, wait_for_finalization=False)

comment_schema = "post_hash STRING,parent_hash STRING,depth NUMERIC,body STRING"
comment_schemaId = addSchema(comment_schema, wait_for_inclusion=True, wait_for_finalization=False)

vote_schema = "post_hash STRING,parent_hash STRING,parent_type STRING,num_votes NUMERIC"
vote_schemaId = addSchema(vote_schema, wait_for_inclusion=True, wait_for_finalization=False)

user_schema = "msa_id NUMERIC,username STRING,password STRING,profile_pic STRING,wallet_ss58_address STRING"
user_schemaId = addSchema(user_schema, wait_for_inclusion=True, wait_for_finalization=False)

follow_schema = "protagonist_msa_id NUMERIC,antagonist_msa_id NUMERIC,event STRING"
follow_schemaId = addSchema(follow_schema, wait_for_inclusion=True, wait_for_finalization=False)

In [37]:
[post_schemaId, comment_schemaId, vote_schemaId, user_schemaId, follow_schemaId]

[7, 8, 3, 4, 5]

# Mint accounts 

In [7]:
msa_keys = {}
for account in ['Alice', 'Charlie', 'Dave', 'Eve', 'Ferdie']:
    wallet = Keypair.create_from_uri(f'//{account}')
    user_msa_id, receipt_user = mint_user(account, "password", "profile_pic", wallet)
    msa_keys[account] = user_msa_id

Extrinsic '0x6f3487322b79a061e5c60f06c243eafa8812ff86db36a80818ac0fe5670e581b' sent and included in block 'None'
Extrinsic '0xee09f22fc082a4f3591ddcebf91b77363f562c80a4ae4573aa3de8a0388a38b9' sent and included in block 'None'
Extrinsic '0x7e09f84547fd0602720eca04409ead3fb1cc633a283098cc1d3ee3c7baadd52a' sent and included in block 'None'
Extrinsic '0xf576f4a1cdf73706b95cfe6d74dff9dc5f3eb8f925e7fcee33cfcaad20eb44d3' sent and included in block 'None'
Extrinsic '0x73c99a4f20a1d075ff77ffe2482abc61286e6b3b5eae4a0f80e43eaf76559003' sent and included in block 'None'


In [8]:
msa_keys

{'Alice': 2, 'Charlie': 3, 'Dave': 4, 'Eve': 5, 'Ferdie': 6}

# Have accounts follow each other

In [9]:
for name1, k1 in msa_keys.items():
    for name2, k2 in msa_keys.items():
        if k1 != k2:
            print(name1, name2)
            receipt = follow_user(k1, k2, True, wait_for_inclusion=False, wait_for_finalization=False)

Alice Charlie
Extrinsic '0x5a88c7f192ce049e29a4f3beafbf90d6cdf7f5e4736b21cfb5339f04a67c2820' sent and included in block 'None'
Alice Dave
Extrinsic '0xb80df0a00e510873d891d7f5abfbc82f73eee6d4f9e1f7145fb9c0105cda5039' sent and included in block 'None'
Alice Eve
Extrinsic '0x1bcf43324fe04c2f3ce00f4c931b5655197dad1fedbd8a0b8977d2d05caae594' sent and included in block 'None'
Alice Ferdie
Extrinsic '0x4600325cdf93696755c9935eb8deb7ae40d08dcb61e52e0ec223dcc1d7ed33d9' sent and included in block 'None'
Charlie Alice
Extrinsic '0xceb0dfd6351a19baaaa81f341d4018d9a836c80b756e85b4d454615d004c7367' sent and included in block 'None'
Charlie Dave
Extrinsic '0x3e6fa5b06da66c006805b800101a1aa80f20f04ba7c8a47c47713b96e2f31b60' sent and included in block 'None'
Charlie Eve
Extrinsic '0xfc1aefea856a9432bfabb5f38c2f9116c1429803104a6fbca1a214efd5c1132e' sent and included in block 'None'
Charlie Ferdie
Extrinsic '0x71f7c4b6f734b1c7589140046f9ca9d0ce7cde205fd944eacf888d774989a53d' sent and included in block '

In [10]:
receipt = follow_user(msa_keys['Dave'], msa_keys['Eve'], False, wait_for_inclusion=False, wait_for_finalization=False)
receipt = follow_user(msa_keys['Eve'], msa_keys['Dave'], False, wait_for_inclusion=False, wait_for_finalization=False)
receipt = follow_user(msa_keys['Dave'], msa_keys['Eve'], True, wait_for_inclusion=False, wait_for_finalization=False)

Extrinsic '0xd749790cde74e27eb2b8d46866a8414331bce4ca09dbec9dd652a80fa3dcf42d' sent and included in block 'None'
Extrinsic '0xdd837d187a14cbabd8c469d89ef3f546595be06257bd132b4cb7cab67e5ed68d' sent and included in block 'None'
Extrinsic '0x14217205ac30aa0932e86106b80a8751a0a8de617d8ff45214edccf170b311da' sent and included in block 'None'


# Test post

In [31]:
%%bash
curl -X 'POST' \
  'http://67.160.97.38:5000/submit/post' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '{
  "category": "string",
  "title": "string",
  "body": "string",
  "url": "http://www.google.com",
  "is_nsfw": true
}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:02:09 --:--:--     0
curl: (28) Failed to connect to 67.160.97.38 port 5000: Connection timed out


CalledProcessError: Command 'b'curl -X \'POST\' \\\n  \'http://67.160.97.38:5000/submit/post\' \\\n  -H \'accept: application/json\' \\\n  -H \'Content-Type: application/json\' \\\n  -d \'{\n  "category": "string",\n  "title": "string",\n  "body": "string",\n  "url": "http://www.google.com",\n  "is_nsfw": true\n}\'\n'' returned non-zero exit status 28.